<a href="https://colab.research.google.com/github/wzhou789/FitnessBody/blob/main/regression_data_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pillow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
directory_path = "/content/drive/My Drive/shared/MLRegression/train"
for filename in os.listdir(directory_path):
    print(filename)

In [ ]:
import re
import xml.etree.ElementTree as ET
from PIL import Image

input_directory = '/content/drive/My Drive/shared/CSV_unprocessed/train/'
new_path = '/content/drive/My Drive/shared/regression/train/Annotations'
imageoutput_directory = '/content/drive/My Drive/shared/regression/train/images'
images_directory = '/content/drive/My Drive/shared/CSV_unprocessed/train/'

In [ ]:
from collections import defaultdict, OrderedDict
import csv
import shutil
import os

annot_path = "/content/drive/My Drive/shared/CSV_unprocessed/train/_annotations.csv"
new_path = "/content/drive/My Drive/shared/regression/train/processed.csv"
new_path_im = "/content/drive/My Drive/shared/regression/train/imgs"

placement = {"Shoulders_Breadth":7, "Shoulders_Symmetry":13, "Shoulders_Tone":19,
            "Chest_Breadth":25, "Chest_Symmetry":31, "Chest_Tone":37,
            "Waist_Breadth":43, "Waist_Symmetry":49, "Waist_Tone":55,
            "Hips_Breadth":61, "Hips_Symmetry":67, "Hips_Tone":73,
            "Legs_Breadth":79, "Legs_Length":85, "Legs_Symmetry":91, "Legs_Tone":97,
            "ArmR_Breadth":103, "ArmR_Length":109, "ArmR_Tone":115,
            "ArmL_Breadth":121, "ArmL_Length":127, "ArmL_Tone":133}

body_parts = {"whole":0, "Shoulders":1, "Chest":2, "Waist":3, "Hips":4,
              "Legs":5, "ArmR":6, "ArmL":7}

record = {}
mapping = defaultdict(list)

#For reference:
#[0 im1, 1 im1_shoulders, 2 im1_chest, 3 im1_waist, 4 im1_hips,
            #5 im1_legs, 6 im1_armL, 7 im1_armR, 8 [x1,y1,x2,y2],
            #9 [0 shoulder_breadth, 1 shoulder_symmetry, 2 shoulder_tone,
            #3 chest_breadth, 4 chest_symmetry, 5 chest_tone,
            #6 waist_breadth, 7 waist_tone, 8 waist_symmetry,
            #9 hips_breadth, 10 hips_tone, 11 hips_symmetry,
            #12 legs_breadth, 13 legs_length, 14 legs_symmetry, 15 legs_tone
            #16 armR_breadth, 17 armR_length, 18 armR_tone,
            #19 armL_breadth, 20 armL_length, 21 armL_tone]]

#What to actually look like
#[0 imagename, width, height, 1 xmin, 2 ymin, 3 xmax, 4 ymax,
#5 filenameSB (part image for Shoulders_Breadth), 6 Shouders_Breadth regression, 7 xminSB, 8 yminSB, 9 xmaxSB, 10 ymaxSB,
#11 filenameSS (part image for Shoulders_Symmetry), 12 Shouders_Symmetry regression, 13 xminSS, 14 yminSS, 15 xmaxSS, 16 ymaxSS,
#17 filenameST (part image for Shoulders_Tone), 18 Shouders_Tone regression, 19 xminST, 20 yminST, 21 xmaxST, 22 ymaxST,
#23 filename chest breadth, 24 cb reg val, 25 xmin 26 27 28,
#29 fn chest sym, 30 reg val, 31, 32, 33, 34,
#35 fn chest tone, 36 val, 37, 38, 39, 40,
#41 fn waist b, 42 val, 43, 44, 45, 46,
#47 fn waist sym, 48 val, 49, 50, 51, 52,
#53 fn waist t, 54 val, 55, 56, 57, 58,
#59 fn hips b, 60 val, 61, 62, 63, 64,
#65 fn hips sym, 66 val, 67, 68, 69, 70,
#71 fn hips tone, 72 val, 73, 74, 75, 76,
#77 fn legs b, 78 val, 79, 80, 81, 82,
#83 fn legs len, 84 val, 85, 86, 87, 88,
#89 fn legs sym, 90 val, 91, 92, 93, 94,
#95 fn legs t, 96 val, 97, 98, 99, 100,
#101 fn armr b, 102 v, 103, 104, 105, 106,
#107 fn armr l, 108 v, 109, 110, 111, 112,
#113 fn armr t, 114 v, 115, 116, 117, 118,
#119 fn arml b, 120 v, 121, 122, 123, 124,
#125 fn arml l, 126 v, 127, 128, 129, 130,
#131 fn arml t, 132 v, 133, 134, 135, 136]


def process_line(line):
    #curr = line.split(",")
    curr = line
    #print(curr)
    img = curr[0][1]
    x1 = int(curr[4][1])
    y1 = int(curr[5][1])
    x2 = int(curr[6][1])
    y2 = int(curr[7][1])
    if img not in record.keys():
        record[img] = [0]*139
    #special case when whole box
    fullimg_path = os.path.join(images_directory,img)
    fullimg = Image.open(fullimg_path)
    if curr[3][1] == "whole":
        record[img][0] = img
        record[img][1] = fullimg.width
        print(fullimg.width)
        record[img][2] = fullimg.height
        record[img][3] = x1
        record[img][4] = y1
        record[img][5] = x2
        record[img][6] = y2
    #any other box
    else:
        regression = curr[3][1].split("_")
        #print(regression)
        reg_val_name = regression[0]+"_"+regression[1]
        if reg_val_name not in placement.keys():
            print(f"Regression value name error in {line}")
        #print(regression)
        reg_val = float(regression[2])
        #print(reg_val_name)
        ind = placement[reg_val_name]
        #print(ind)
        record[img][ind] = reg_val_name
        record[img][ind+1] = reg_val
        record[img][ind+2] = curr[4][1]
        record[img][ind+3] = curr[5][1]
        record[img][ind+4] = curr[6][1]
        record[img][ind+5] = curr[7][1]
        part = body_parts[regression[0]]
        part_filename = os.path.splitext(img)[0]+"_"+reg_val_name+os.path.splitext(img)[1]
        #crop_img = create_img(img, reg_val_name, x1, y1, x2, y2)
        record[img][placement[reg_val_name]] = part_filename

    #record[(img,reg_val_name)] = [crop_img,reg_val_name,reg_val,x1,y1,x2,y2]
    #Map to easily access image name in record key
    #mapping[img].append((img, reg_val_name))

def create_img(img, reg_val_name, x1, y1, x2, y2):
    im_path = os.path.splitext(img)[0]+"_"+reg_val_name+os.path.splitext(img)[1]
    colab_img = os.path.join(input_directory, img)
    im2 = shutil.copy(colab_img, im_path)
    im = Image.open(im2)
    new_im = im.crop((x1,y1, x2, y2))
    return new_im.save(os.path.join(new_path_im,im_path), 'JPEG')

def main():
    filename = new_path
    with open(annot_path) as csvfile:
        data = csv.DictReader(csvfile)
        for row in data:
            new_row = list(row.items())
            process_line(new_row)
    print(new_path)
    with open(new_path, 'w', newline='') as newfile:
        writer = csv.writer(newfile)
        writer.writerow(['wholefilename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax',
                         'filenameSB', 'Shoulder_Breadth', 'xminSB', 'yminSB', 'xmaxSB', 'ymaxSB',
                         'filenameSS', 'Shoulder_Symmetry','xminSS', 'yminSS', 'xmaxSS', 'ymaxSS',
                         'filenameST', 'Shoulder_Tone', 'xminST', 'yminST', 'xmaxST', 'ymaxST',
                         'filenameCB', 'Chest_Breadth', 'xminCB', 'yminCB', 'xmaxCB', 'ymaxCB',
                         'filenameCS', 'Chest_Symmetry', 'xminCS', 'yminCS', 'xmaxCS', 'ymaxCS',
                         'filenameCT', 'Chest_Tone', 'xminCT', 'yminCT', 'xmaxCT', 'ymaxCT',
                         'filenameWB', 'Waist_Breadth', 'xminWB', 'yminWB', 'xmaxWB', 'ymaxWB',
                         'filenameWS', 'Waist_Symmetry', 'xminWS', 'yminWS', 'xmaxWS', 'ymaxWS',
                         'filenameWT', 'Waist_Tone', 'xminWT', 'yminWT', 'xmaxWT', 'ymaxWT',
                         'filenameHB', 'Hips_Breadth', 'xminHB', 'yminHB', 'xmaxHB', 'ymaxHB',
                         'filenameHS', 'Hips_Symmetry', 'xminHS', 'yminHS', 'xmaxHS', 'ymaxHS',
                         'filenameHT', 'Hips_Tone', 'xminHT', 'yminHT', 'xmaxHT', 'ymaxHT',
                         'filenameLB', 'Legs_Breadth', 'xminLB', 'yminLB', 'xmaxLB', 'ymaxLB',
                         'filenameLS', 'Legs_Symmetry', 'xminLS', 'yminLS', 'xmasLS', 'ymaxLS',
                         'filenameLT', 'Legs_Tone', 'xminLT', 'yminLT', 'xmaxLT', 'ymaxLT',
                         'filenameALB', 'ArmL_Breadth', 'xminALB', 'yminALB', 'xmaxALB', 'ymaxALB',
                         'filenameALL', 'ArmL_Length', 'xminALL', 'yminALL', 'xmaxALL', 'ymaxALL',
                         'filenameALT', 'ArmL_Tone', 'xminALT', 'yminALT', 'xmaxALT', 'ymaxALT',
                         'filenameARB', 'ArmR_Breadth', 'xminARB', 'yminARB', 'xmaxARB', 'ymaxARB',
                         'filenameARL', 'ArmR_Length', 'xminARL', 'yminARL', 'xmaxARL', 'ymaxARL',
                         'filenameART', 'ArmR_Tone', 'xminART', 'yminART', 'xmaxART', 'ymaxART',
                         ])
        for key in record:
            writer.writerow(record[key])
            #print(record[key])
    with open(new_path) as csvfile:
      data = csv.DictReader(csvfile)
      for row in data:
        print(row)

    print("Finished running")

if __name__=="__main__":
    main()